###  Seleção de variáveis mais relevantes
* Randon Forest => Gera vários inputs de dados a partir de um método randomico e cria uma árvore binaria para cada um deles, realizando assim a predição com os melhores resultados (É possível treinar esse modelo e verificar quais foram as variaveis mais importantes para ele). 
* Fisher => Calcula o escore de Fisher para selecionar as caracteristicas mais importantes de um conjunto de dados, basicamente ele realiza a divizão do "erro médio quadratico ponderado" pela "variância ponderada" (Essa implementação foca em algoritmos de classificação com classes definidas por inteiros).
* Gini => calcula o índice de impureza de Gini para cada feature (variável independente) em relação à variável dependente Y. O índice de impureza de Gini é uma medida de quão impura é a divisão das amostras em duas classes, e é usado em árvores de decisão para selecionar o melhor atributo para dividir os dados em subconjuntos mais homogêneos (Utilizado em bases de dados definidas para classificação com classes referênciadas em int).
* KruskalWallis => É um teste não-paramétrico que compara as medianas de duas ou mais amostras independentes para determinar se há diferenças estatisticamente significativas entre elas (Pega os valores em que a maior próximidade estatistica -> não implica diretamente que possa ser uma boa escolha para o modelo).
* Lasso => (Least Absolute Shrinkage and Selection Operator) é um método de regressão linear que utiliza a regularização L1 para selecionar as variáveis mais importantes em um conjunto de dados. O objetivo do Lasso é minimizar a soma dos erros quadrados (Caráter linear).
* ElasticNet => é uma extensão do algoritmo Lasso (Least Absolute Shrinkage and Selection Operator) e Ridge Regression, que combina a penalização L1 e L2 para selecionar variáveis importantes em um modelo de regressão linear. É utilizado para lidar com problemas de regressão em que há um grande número de variáveis preditoras (high-dimensional data) e muitas dessas variáveis podem não ser relevantes para a predição da variável resposta (Caráter linear).
* Chi2 ($X^2$) =>  é um teste estatístico que é usado para determinar se existe uma relação significativa entre duas variáveis categóricas. Ele é usado para avaliar se as diferenças entre as frequências observadas e as frequências esperadas são significativas o suficiente para rejeitar a hipótese nula de que não há relação entre as duas variáveis.
* F_Regression => Ele utiliza a estatística F de Fisher para avaliar a relação linear entre as variáveis e calcular o p-valor associado a cada feature.
* Mutual_Info_Regression => É uma técnica de seleção de recursos que mede a dependência mútua entre cada recurso e a variável de destino, usando a entropia da informação. Ele estima a informação mútua entre cada recurso e o destino, para ajudar na seleção de recursos que são relevantes para a predição do alvo.

In [1]:
# Import das bibliotecas utilizadas

from sklearn.feature_selection import SelectKBest, chi2, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet, Lasso
from scipy.stats import kruskal
from enum import Enum 

import pandas as pd
import numpy as np


class methods(Enum):
    # Modelos de previsão 
    RandomForest = 1               # Classificação / Regressão
    Lasso = 2                      # Classificação / Regressão
    ElasticNet = 3                 # Classificação / Regressão
    
    # Métodos estatísticos
    FsFisher = 4                   # Classificação (class em int -> int(y))
    FsGini = 5                     # Classificação (class em int -> int(y))
    KruskalWallis = 6              # Classificação / Regressão
    Chi2 = 7                       # Classificação (class em int -> int(y))
    FRegression = 8                # Regressão
    MutualRegression = 9           # Regressão

In [2]:
# Configurações globais do processo de seleção
inputDataName = "PETR4_B_0_30min"


usedMethodsD1  = [methods.FsFisher, methods.FsGini, methods.Chi2, methods.RandomForest];
usedMethodsD2  = [methods.MutualRegression, methods.Lasso, methods.ElasticNet, methods.RandomForest];

# usedMethodsD1  = [methods.Chi2];
# usedMethodsD2  = [];

# usedMethodsD1  = [methods.FsGini];
# usedMethodsD2  = [methods.FsGini];

usedMethodsDB1 =  methods.KruskalWallis;
usedMethodsDB2 =  methods.KruskalWallis;

filterEachD1   = [20, 20, 20, 20];
filterEachD2   = [20, 20, 20, 20];
dataset1_2     = [4, 4];

# RandomForest
treesqtd = 200;

#Lasso
lassoAlpha = 0.05

#ElasticNet
elasticAlpha  = 0.5
elasticRation = 0.5

In [3]:
# Inserção dos dados para seleção de variáveis
X = pd.read_csv(f'Generated/In_{inputDataName}.csv',  sep=";") ;
Y = pd.read_csv(f'Generated/Out_{inputDataName}.csv', sep=";") ;

# Configurando as opções de exibição
pd.set_option('display.max_columns', None)  # Exibir todas as colunas
pd.set_option('display.width', None)  # Permitir exibição completa sem quebrar a linha

# print(X.isna().sum().to_string())
# print(Y.isna().sum().to_string())


display(X.head(3));
display(Y.head(3));

,Abertura_SMA_3,Abertura_SMA_5,Abertura_SMA_15,Abertura_SMA_30,Abertura_SMA_200,Máximo_SMA_3,Máximo_SMA_5,Máximo_SMA_15,Máximo_SMA_30,Máximo_SMA_200,Mínimo_SMA_3,Mínimo_SMA_5,Mínimo_SMA_15,Mínimo_SMA_30,Mínimo_SMA_200,Fechamento_SMA_3,Fechamento_SMA_5,Fechamento_SMA_15,Fechamento_SMA_30,Fechamento_SMA_200,Abertura_EMA_5,Abertura_EMA_7,Abertura_EMA_9,Abertura_EMA_12,Abertura_EMA_26,Máximo_EMA_5,Máximo_EMA_7,Máximo_EMA_9,Máximo_EMA_12,Máximo_EMA_26,Mínimo_EMA_5,Mínimo_EMA_7,Mínimo_EMA_9,Mínimo_EMA_12,Mínimo_EMA_26,Fechamento_EMA_5,Fechamento_EMA_7,Fechamento_EMA_9,Fechamento_EMA_12,Fechamento_EMA_26,Abertura_MACD_4:22,Abertura_MACD_8:17,Abertura_MACD_12:26,Máximo_MACD_4:22,Máximo_MACD_8:17,Máximo_MACD_12:26,Mínimo_MACD_4:22,Mínimo_MACD_8:17,Mínimo_MACD_12:26,Fechamento_MACD_4:22,Fechamento_MACD_8:17,Fechamento_MACD_12:26,HLC_CCI_14,HLC_CCI_17,HLC_CCI_18,HLC_CCI_20,ADX_7,ADX_14,Abertura_MTM_5,Abertura_MTM_10,Abertura_MTM_11,Abertura_MTM_12,Máximo_MTM_5,Máximo_MTM_10,Máximo_MTM_11,Máximo_MTM_12,Mínimo_MTM_5,Mínimo_MTM_10,Mínimo_MTM_11,Mínimo_MTM_12,Fechamento_MTM_5,Fechamento_MTM_10,Fechamento_MTM_11,Fechamento_MTM_12,Abertura_TSI_13:25,Máximo_TSI_13:25,Mínimo_TSI_13:25,Fechamento_TSI_13:25,K%_8,K%_10,K%_14,K%_14_SMA_3,R%_21,R%_14,R%_5,Abertura|T-7|,Máximo|T-7|,Mínimo|T-7|,Fechamento|T-7|,Abertura|T-6|,Máximo|T-6|,Mínimo|T-6|,Fechamento|T-6|,Abertura|T-5|,Máximo|T-5|,Mínimo|T-5|,Fechamento|T-5|,Abertura|T-4|,Máximo|T-4|,Mínimo|T-4|,Fechamento|T-4|,Abertura|T-3|,Máximo|T-3|,Mínimo|T-3|,Fechamento|T-3|,Abertura|T-2|,Máximo|T-2|,Mínimo|T-2|,Fechamento|T-2|,Abertura|T-1|,Máximo|T-1|,Mínimo|T-1|,Fechamento|T-1|,Abertura,Máximo,Mínimo,Fechamento
0,12.883333,12.824,12.504000,12.393667,11.88985,12.920000,12.872,12.564000,12.461667,11.9374,12.750000,12.614,12.378667,12.311667,11.83885,12.776667,12.644,12.430000,12.366000,11.88050,12.801822,12.739054,12.686069,12.623227,12.457186,12.854208,12.795908,12.745584,12.684991,12.520485,12.659992,12.593579,12.543240,12.487492,12.349227,12.704792,12.639841,12.590876,12.536554,12.399785,0.341853,0.162800,0.166041,0.327488,0.157839,0.164506,0.321762,0.142025,0.138265,0.315850,0.138766,0.136769,128.350194,128.704016,132.284805,141.449950,0.303394,0.262475,4.707792,4.792851,4.622871,4.538088,3.272147,4.777328,4.692557,4.186795,4.905969,4.224208,4.224208,4.478827,4.112903,4.703974,4.619125,4.534413,-30.988042,-32.446629,-28.619163,-29.082347,4.666667,15.00,4.333333,4.688034,-6.666667,-3.333333,-1.266667,12.46,12.52,12.35,12.42,12.40,12.49,12.36,12.46,12.32,12.53,12.23,12.40,12.56,12.66,12.27,12.32,12.91,12.94,12.55,12.57,12.84,12.85,12.58,12.58,12.91,12.97,12.84,12.84,12.90,12.94,12.83,12.91
1,12.910000,12.896,12.556000,12.408333,11.89640,12.950000,12.928,12.612667,12.475667,11.9436,12.853333,12.738,12.438000,12.326667,11.84525,12.883333,12.760,12.482667,12.379333,11.88650,12.841214,12.784290,12.732855,12.668884,12.491468,12.882805,12.831931,12.784467,12.724223,12.551560,12.736661,12.667684,12.612592,12.549416,12.389284,12.769861,12.704881,12.652701,12.592469,12.436838,0.338514,0.167897,0.177416,0.316261,0.159227,0.172663,0.352795,0.162160,0.160132,0.336538,0.155152,0.155631,111.240463,115.302519,116.620530,124.138351,0.246474,0.237859,2.866242,4.446241,4.955321,4.785077,2.211690,4.439064,4.777328,4.692557,5.052975,4.882018,4.711617,4.711617,4.707792,4.792851,4.622871,4.538088,-32.892885,-33.960920,-32.488837,-32.466188,3.235294,6.10,3.347826,4.637310,-11.600000,-2.347826,-0.615385,12.40,12.49,12.36,12.46,12.32,12.53,12.23,12.40,12.56,12.66,12.27,12.32,12.91,12.94,12.55,12.57,12.84,12.85,12.58,12.58,12.91,12.97,12.84,12.84,12.90,12.94,12.83,12.91,12.92,12.94,12.89,12.90
2,12.876667,12.876,12.586667,12.420000,11.90240,12.933333,12.924,12.650000,12.489667,11.9499,12.843333,12.790,12.483333,12.338667,11.85130,12.910000,12.830,12.534667,12.394000,11.89305,12.830810,12.790718,12.748284,12.690594,12.515063,12.895204,12.853948,12.811574,12.754343,12.578852,12.761108,12.703263,12.652073,12.589506,12.420448,12.81

,OutPut |T+1|,OutPut_class |T+1|
0,12.90,0
1,12.92,1
2,12.82,0


#### Funções de seleção

In [4]:
# Funções de seleção de variáveis

# ****************************************** RandomForest ****************************************
def randomForest(X,Y,qtdF):
    model = RandomForestRegressor(n_estimators=treesqtd, random_state=42);
    model.fit(X, Y.values.ravel());
    importances = pd.DataFrame({'feature': X.columns, 'importance':model.feature_importances_});
    importances = importances.sort_values('importance', ascending=False);
    usedVars = importances.index[0:qtdF]
    return X.iloc[:,usedVars];

# ********************************************* Fisher *******************************************
def fisher(X, Y, qtdF):
    #X, Y = X.iloc[:10,:], Y.iloc[:10]
    colq = X.shape[1]
    maxY = int(np.max(Y)+1)
    minY = int(np.min(Y))
    out  = pd.DataFrame({'W': np.zeros(colq)})
    classIdx = [np.where(Y==j)[0] for j in range(minY, maxY)]
    classQtd = [len(classIdx[j])  for j in range(maxY-minY)]
    for i in range(colq):
        errMed  = 0
        varPon  = 0
        col     = X.iloc[:,i]
        colMean = np.mean(col)
        for j in range(maxY-minY):
            classMean = np.mean(col.iloc[classIdx[j]])
            classVari = np.var(col.iloc[classIdx[j]], ddof=1)
            errMed   += classQtd[j] * (classMean-colMean)**2
            varPon   += classVari * classQtd[j]
        if errMed == 0:
            out['W'][i] = 0
        elif varPon == 0:
            out['W'][i] = 100
        else:
            out['W'][i] = errMed/varPon       
    return X.iloc[:,(out['W'].sort_values(ascending=False).index)[:qtdF]]
      
# ********************************************* Gine ********************************************
def gini(X, Y, qtd):
    a, n = X.shape
    W = np.zeros((n, 1))

    for i in range(n):
        values = np.unique(X.iloc[:, i])
        v = len(values)
        W[i] = 0.5
        for j in range(v):
            left_Y = Y[X.iloc[:, i] <= values[j]]
            right_Y = Y[X.iloc[:, i] > values[j]]

            gini_left = 0
            gini_right = 0

            if len(left_Y) > 0:
                for k in range(np.min(Y), np.max(Y) + 1):
                    gini_left += (len(left_Y[left_Y == k]) / len(left_Y))**2
                gini_left = 1 - gini_left

            if len(right_Y) > 0:
                for k in range(np.min(Y), np.max(Y) + 1):
                    gini_right += (len(right_Y[right_Y == k]) / len(right_Y))**2
                gini_right = 1 - gini_right

            current_gini = 0
            if len(Y) > 0:
                current_gini = (len(left_Y) * gini_left + len(right_Y) * gini_right) / len(Y)

            if current_gini < W[i]:
                W[i] = current_gini
                       
    W = np.sort(W, axis=0)[::-1].flatten(), np.argsort(W, axis=0)[::-1].flatten()

    return X.iloc[:,W[1]]


# ***************************************** KruskalWallis ***************************************
def kruskalWallis(X, Y, qtd):
    n = X.shape[1]
    out = pd.DataFrame(np.zeros(n), columns=["W"])
    for i in range(n):
        out['W'].iloc[i] = - np.mean(kruskal(X.iloc[:,i], Y.iloc[:])[1])
    out = out.sort_values('W', ascending=False).index
    return X.iloc[:,out[:qtd]]

# ******************************************* Lasso *********************************************
def lasso(data, Y, qtd):
    model = Lasso(alpha=lassoAlpha, random_state=0)
    model.fit(data, Y)
    rate = pd.DataFrame(abs(model.coef_), columns=["W"])
    rate = rate.sort_values('W', ascending=False).index    
    return data.iloc[:, rate[:qtd]]

# **************************************** ElasticNet *******************************************
def elasticNet(data, Y, qtd):
    model = ElasticNet(alpha=elasticAlpha, l1_ratio=elasticRation)
    model.fit(data, Y)
    rate = pd.DataFrame(abs(model.coef_), columns=["W"])
    rate = rate.sort_values('W', ascending=False).index    
    return data.iloc[:, rate[:qtd]]

# ****************************************** Chi2 (X²) *******************************************
def ftChi2(data, Y, qtd):
    selector = pd.DataFrame(chi2(data.abs(), Y.values.ravel())[0], columns=["W"])
    selector = selector.sort_values('W', ascending=False).index
    return data.iloc[:, selector[:qtd]];

# *************************************** F Regression ******************************************
def fRegression(data, Y, qtd):
    selector = pd.DataFrame(f_regression(data, Y.values.ravel())[0], columns=["W"])
    selector = selector.sort_values('W', ascending=False).index
    return data.iloc[:, selector[:qtd]];

# *********************************** Mutual Info Regression*************************************
def mutualRegression(data, Y, qtd):
    mi = mutual_info_regression(data, Y.values.ravel())
    selector = pd.DataFrame(mi, columns=["W"])
    selector = selector.sort_values('W', ascending=False).index
    return data.iloc[:, selector[:qtd]];
    
# ************************************** seleção dos dados **************************************

def select(data, Y, qtd, option):
    if(option == methods.RandomForest):
        data = randomForest(data,Y.iloc[:,0],qtd); 
        
    elif(option == methods.FsFisher):
        data = fisher(data,Y.iloc[:,1],qtd); 
        
    elif(option == methods.FsGini):
        data = gini(data,Y.iloc[:,1],qtd); 
        
    elif(option == methods.KruskalWallis):
        data = kruskalWallis(data,Y.iloc[:,0],qtd);
        
    elif(option == methods.Lasso):
        data = lasso(data,Y.iloc[:,0],qtd);
        
    elif(option == methods.ElasticNet):
        data = elasticNet(data,Y.iloc[:,0],qtd);
        
    elif(option == methods.Chi2):
        data = ftChi2(data,Y.iloc[:,1],qtd);
        
    elif(option == methods.FRegression):
        data = fRegression(data,Y.iloc[:,0],qtd);
        
    elif(option == methods.MutualRegression):
        data = mutualRegression(data,Y.iloc[:,0],qtd);
    return data;

def generate(data, Y, opt1, qtd1, opt2, qtd2):
    
    dataset1 = pd.DataFrame({})
    dataset2 = pd.DataFrame({})
    
    print("Gerando dataset1...")
    for i in range(len(opt1)):
        print("        ->",opt1[i])
        dataset1 = dataset1.combine_first(select(data, Y, qtd1[i], opt1[i]))
    
    print("done.")
    print("Gerando dataset2...")
    for i in range(len(opt2)):
        print("        ->",opt2[i])
        dataset2 = dataset2.combine_first(select(data, Y, qtd2[i], opt2[i]))
    
    print("done.")
    
    dataset1 = select(dataset1, Y, dataset1_2[0], usedMethodsDB1)
    dataset2 = select(dataset2, Y, dataset1_2[1], usedMethodsDB2)
    
    
    return [dataset1, dataset2]
    

In [5]:
dataset1, dataset2 = generate(X, Y, usedMethodsD1, filterEachD1, usedMethodsD2, filterEachD2)

display(dataset1.head(3))
display(dataset2.head(3))

Gerando dataset1...
        -> methods.FsFisher
        -> methods.FsGini
        -> methods.Chi2
        -> methods.RandomForest
done.
Gerando dataset2...
        -> methods.MutualRegression
        -> methods.Lasso


/home/souza/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.658e+00, tolerance: 2.625e+00
  model = cd_fast.enet_coordinate_descent(


        -> methods.ElasticNet


/home/souza/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.343e+01, tolerance: 2.625e+00
  model = cd_fast.enet_coordinate_descent(


        -> methods.RandomForest
done.


,ADX_14,Fechamento_MACD_12:26,Fechamento_MACD_8:17,Fechamento_MTM_10
0,0.262475,0.136769,0.138766,4.703974
1,0.237859,0.155631,0.155152,4.792851
2,0.217097,0.170231,0.164872,4.446241


,ADX_14,Fechamento_MACD_12:26,R%_21,R%_14
0,0.262475,0.136769,-6.666667,-3.333333
1,0.237859,0.155631,-11.600000,-2.347826
2,0.217097,0.170231,-3.117647,-3.571429


In [6]:
dataset1.to_csv(f'Selected/In_class_{inputDataName}.csv', index=False, sep = ';')  
dataset2.to_csv(f'Selected/In_regre_{inputDataName}.csv', index=False, sep = ';')

Y.to_csv(f'Selected/{inputDataName}_Out.csv', index=False, sep = ';') 